In [7]:
import numpy as np
import imageio

def array_to_list(array):
    L = []
    for i in range(len(array)):
        for j in range(len(array)):
            L.append(array[i,j])
    return L    

letter_array =['B','C','D','E','F','G','H','I','J','K'];
base_ref = []
label_ref = []
for letter in range(0,10):
    for i in range(1,151):
        image_file_name = 'data_resized/' + letter_array[letter] + str(i) + '.bmp'
        image = imageio.imread(image_file_name)
        list_img = array_to_list(image)
        base_ref.append(list_img)
        label_ref.append(letter_array[letter])
label_ref = np.array([e for e in label_ref])       
base_ref = np.array([l for l in base_ref])

base_val = []
label_val = []
for letter in range(0,10):
    for i in range(151,201):
        image_file_name = 'data_resized/' + letter_array[letter] + str(i) + '.bmp'
        image = imageio.imread(image_file_name)
        list_img = array_to_list(image)
        base_val.append(list_img)
        label_val.append(letter_array[letter])
        
label_val = np.array([e for e in label_val])       
base_val = np.array([l for l in base_val])

base_test = []
label_test = []
for letter in range(0,10):
    for i in range(201,251):
        image_file_name = 'data_resized/' + letter_array[letter] + str(i) + '.bmp'
        image = imageio.imread(image_file_name)
        list_img = array_to_list(image)
        base_test.append(list_img)
        label_test.append(letter_array[letter])
        
label_test = np.array([e for e in label_test])       
base_test = np.array([l for l in base_test])

In [8]:
def base_moyenne(base):
    res = []
    classes = []
    for i in range(10):
        classe = []
        avg = base_ref[i*150,:]
        j = 0
        while j < 150:
            k = i*150 + j
            j += 1
            avg = [x + y for x, y in zip(avg, base_ref[k,:])]
            classe.append(base_ref[k,:])
        avg = [int(e/50) for e in avg]
        res.append(avg)
        classes.append(classe)
    return (res,classes) 

In [9]:
def iv(k):
    cov_matrix = np.cov(np.transpose(base_moyenne(base_ref)[1][k]))
    diag = np.array([0.01]*144)
    diag = np.diag(diag)
    cov_matrix = cov_matrix + diag
    iv = np.linalg.inv(cov_matrix)
    return iv

In [4]:
from scipy.spatial import distance

def test_ppv(base_ref,label_ref,image,iv_array):
    dist_image_ref = []
    for i in range(len(base_ref)):
        dist = np.linalg.norm(image - base_ref[i])
        dist_image_ref.append(dist)
    i = np.argmin(dist_image_ref)
    class_min_dist = label_ref[i]
    return class_min_dist

In [10]:
#mahalanobis
from scipy.spatial import distance

def test_ppv(base_ref,label_ref,image,iv_array):
    dist_image_ref = []
    for i in range(len(base_ref)):
        dist = distance.mahalanobis(image,base_ref[i],iv_array[i])
        dist_image_ref.append(dist)
    i = np.argmin(dist_image_ref)
    class_min_dist = label_ref[i]
    return class_min_dist

In [11]:
def label_pred(base,base_ref,label_ref):
    label_pred = []
    iv_array = [iv(i) for i in range(0,10)]
    for i in range(len(base)):
        pred = test_ppv(base_ref,label_ref,base[i,:],iv_array)
        label_pred.append(pred)
    return label_pred

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

label_red = np.array(letter_array)
base_red = base_moyenne(base_ref)[0]

print(confusion_matrix(label_test, label_pred(base_test,base_red,label_red)))

/home/thoreau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in ubyte_scalars
  # This is added back by InteractiveShellApp.init_path()


[[28  0 12  1  0  3  1  0  1  4]
 [ 0 35  1  5  0  8  1  0  0  0]
 [10  0 35  0  0  3  1  0  1  0]
 [ 1  3  0 38  0  5  0  0  0  3]
 [ 1  0  0 11 31  1  0  0  1  5]
 [ 1  0  0  2  1 45  0  0  0  1]
 [ 1  0  3  0  0  2 35  0  1  8]
 [ 3  0  2  3  1  1  1 23 11  5]
 [ 0  0  1  1  0  1  1  0 45  1]
 [ 0  0  0  1  0  1  0  0  0 48]]
